In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
def db(x):
    """ Convert linear value to dB value """
    return 10*np.log10(x)

import blimpy as bl

import sys
sys.path.insert(0, "/home/bryanb/setigen/")
import setigen as stg

bshuf filter already loaded, skip it.
lz4 filter already loaded, skip it.
zfp filter already loaded, skip it.


In [2]:
2**25

33554432

In [7]:
r'''
Synthesize a Filterbank file.
Motivation: https://github.com/UCBerkeleySETI/turbo_seti/issues/206
'''

import time
from astropy import units as u
import setigen as stg
from blimpy import Waterfall


DIR = '/datax/scratch/bbrzycki/data/'
INFILE = '/datax/scratch/bbrzycki/data/tabby.1.fil'
MAX_DRIFT = 4
MIN_SNR = 25
arg_n_signals = 1
 
    
def gen_fil(arg_path, n_signals):
    r''' Generate a Filterbank file '''

    assert n_signals > 0 and n_signals < 5
    
    # Define parameters.
    
#     fchans = 134217728
    fchans = 2**26
    #fchans = 1342177
    tchans = 5
    df = -2.7939677238464355e-06 * u.Hz
    dt = 18.253611008 * u.s
    fch1 = 19626.464842353016 * u.MHz
    noise_std = 0.05 # Gaussian standard deviation
    
    sig_snr_1 = 100.0
    sig_width_1 = 1.1 * u.Hz
    drate_1 = 1.6 * u.Hz/u.s
    f_start_1 = 0

    sig_snr_2 = 200.0
    sig_width_2 = 1.2 * u.Hz
    drate_2 = 1.3 * u.Hz/u.s
    f_start_2 = fchans * 0.1

    sig_snr_3 = 300.0
    sig_width_3 = 1.3 * u.Hz
    drate_3 = 2.6 * u.Hz/u.s
    f_start_3 = fchans * 0.2

    sig_snr_4 = 400.0
    sig_width_4 = 1.4 * u.Hz
    drate_4 = 3.2 * u.Hz/u.s
    f_start_4 = fchans * 0.3
    
    # Generate the frame.
    frame = stg.Frame(fchans=fchans,
                      tchans=tchans,
                      df=df,
                      dt=dt,
                      fch1=fch1)
    
    # Add noise.
    frame.add_noise(x_mean=0, x_std=noise_std, noise_type='gaussian')

    # Add signal 1.
    signal_intensity = frame.get_intensity(snr=sig_snr_1)
    frame.add_constant_signal(f_start=frame.get_frequency(f_start_1),
                              drift_rate=drate_1,
                              level=signal_intensity,
                              width=sig_width_1,
                              f_profile_type='gaussian')
    
    # Add signal 2.
    if n_signals > 1:    
        signal_intensity = frame.get_intensity(snr=sig_snr_2)
        frame.add_constant_signal(f_start=frame.get_frequency(f_start_2),
                                  drift_rate=drate_2,
                                  level=signal_intensity,
                                  width=sig_width_2,
                                  f_profile_type='gaussian')
    
    # Add signal 3.
    if n_signals > 2:    
        signal_intensity = frame.get_intensity(snr=sig_snr_3)
        frame.add_constant_signal(f_start=frame.get_frequency(f_start_3),
                                  drift_rate=drate_3,
                                  level=signal_intensity,
                                  width=sig_width_3,
                                  f_profile_type='gaussian')
    
    # Add signal 4.
    if n_signals > 3:    
        signal_intensity = frame.get_intensity(snr=sig_snr_4)
        frame.add_constant_signal(f_start=frame.get_frequency(f_start_4),
                                  drift_rate=drate_4,
                                  level=signal_intensity,
                                  width=sig_width_4,
                                  f_profile_type='gaussian')
    
    # Save Filterbank file.
    frame.save_fil(arg_path)


t1 = time.time()
gen_fil(INFILE, n_signals=1)
t2 = time.time()
print('gen_fil elapsed time = {:.1f}s'.format(t2 - t1))
wf = Waterfall(INFILE)
wf.info()


ValueError: cannot reshape array of size 32768 into shape (0,1,67108864)

In [8]:
!ls /datax/scratch/bbrzycki/data/tabby.1.fil

/datax/scratch/bbrzycki/data/tabby.1.fil


In [10]:
wf = Waterfall(INFILE)
wf.info()


--- File Info ---
      machine_id :                               20
    telescope_id :                                6
         src_raj :                         17:47:15
         src_dej :                     -28:22:59.16
        az_start :                              0.0
        za_start :                              0.0
       data_type :                                1
            fch1 :           19626.464842353016 MHz
            foff :      -2.7939677238464356e-12 MHz
          nchans :                         67108864
          nbeams :                                1
           ibeam :                               -1
           nbits :                               32
   tstart (ISOT) :          2018-12-13T17:12:37.000
    tstart (MJD) :               58465.717094907406
           tsamp :                     18.253611008
            nifs :                                1
     source_name :                        Synthetic
     rawdatafile :                        Syn

In [12]:
fchans = 1024
tchans = 5
df = -2.7939677238464355e-06 * u.Hz
dt = 18.253611008 * u.s
fch1 = 19626.464842353016 * u.MHz
noise_std = 0.05 # Gaussian standard deviation

# Generate the frame.
frame = stg.Frame(fchans=fchans,
                  tchans=tchans,
                  df=df,
                  dt=dt,
                  fch1=fch1)

In [15]:
frame.get_waterfall()

Waterfall data: /home/bryanb/setigen/setigen/assets/sample.fil

In [20]:
vars(frame.waterfall.container)

{'t_begin': 0,
 't_end': 5,
 'header_keywords_types': {'telescope_id': '<l',
  'machine_id': '<l',
  'data_type': '<l',
  'barycentric': '<l',
  'pulsarcentric': '<l',
  'nbits': '<l',
  'nsamples': '<l',
  'nchans': '<l',
  'nifs': '<l',
  'nbeams': '<l',
  'ibeam': '<l',
  'rawdatafile': 'str',
  'source_name': 'str',
  'az_start': '<d',
  'za_start': '<d',
  'tstart': '<d',
  'tsamp': '<d',
  'fch1': '<d',
  'foff': '<d',
  'refdm': '<d',
  'period': '<d',
  'src_raj': 'angle',
  'src_dej': 'angle'},
 'filename': '/home/bryanb/setigen/setigen/assets/sample.fil',
 'load_data': True,
 'header': {'machine_id': 20,
  'telescope_id': 6,
  'src_raj': <Angle 17.7875 hourangle>,
  'src_dej': <Angle -28.3831 deg>,
  'az_start': 0.0,
  'za_start': 0.0,
  'data_type': 1,
  'fch1': 19626.464842353016,
  'foff': -2.7939677238464356e-12,
  'nchans': 1024,
  'nbeams': 1,
  'ibeam': -1,
  'nbits': 32,
  'tstart': 58465.717094907406,
  'tsamp': 18.253611008,
  'nifs': 1,
  'source_name': 'Synthetic'

In [83]:
vars(frame.waterfall)

{'filename': '/home/bryanb/setigen/setigen/assets/sample.fil',
 'ext': '.fil',
 'container': <blimpy.io.fil_reader.FilReader at 0x7fd7bc049a90>,
 'file_header': {'machine_id': 20,
  'telescope_id': 6,
  'src_raj': <Angle 17.7875 hourangle>,
  'src_dej': <Angle -28.3831 deg>,
  'az_start': 0.0,
  'za_start': 0.0,
  'data_type': 1,
  'fch1': 19626.464842353016,
  'foff': -2.7939677238464355e-06,
  'nchans': 67108864,
  'nbeams': 1,
  'ibeam': -1,
  'nbits': 32,
  'tstart': 58465.717094907406,
  'tsamp': 18.253611008,
  'nifs': 1,
  'source_name': 'Synthetic',
  'rawdatafile': 'Synthetic'},
 'header': {'machine_id': 20,
  'telescope_id': 6,
  'src_raj': <Angle 17.7875 hourangle>,
  'src_dej': <Angle -28.3831 deg>,
  'az_start': 0.0,
  'za_start': 0.0,
  'data_type': 1,
  'fch1': 19626.464842353016,
  'foff': -2.7939677238464355e-06,
  'nchans': 67108864,
  'nbeams': 1,
  'ibeam': -1,
  'nbits': 32,
  'tstart': 58465.717094907406,
  'tsamp': 18.253611008,
  'nifs': 1,
  'source_name': 'Syn

In [22]:
wf = frame.get_waterfall()

chunk_dim = wf._get_chunk_dimensions()
blob_dim = wf._get_blob_dimensions(chunk_dim)
n_blobs = wf.container.calc_n_blobs(blob_dim)

chunk_dim, blob_dim, n_blobs

((1, 1, 1048576), (5, 1, 1024), 1)

In [23]:
frame.waterfall.container

In [24]:
frame.waterfall.container.calc_n_blobs(blob_dim)

1

In [25]:
blob_dim[frame.waterfall.container.time_axis]

5

In [26]:
frame.waterfall.container.selection_shape

(5, 1, 1024)

In [28]:
updated_blob_dim=(int(frame.waterfall.container.selection_shape[frame.waterfall.container.time_axis] - blob_dim[frame.waterfall.container.time_axis]*1),
 1,
 int(blob_dim[frame.waterfall.container.freq_axis]))
updated_blob_dim

(0, 1, 1024)

In [29]:
np.prod(updated_blob_dim)

0

In [30]:
frame.waterfall.container._d_type

'float32'

In [31]:
frame.waterfall.container.filename

'/home/bryanb/setigen/setigen/assets/sample.fil'

In [33]:
frame1 = stg.Frame('/home/bryanb/setigen/setigen/assets/sample.fil')
np.prod(frame1.shape)

32768

In [35]:
frame.waterfall.container.MAX_DATA_ARRAY_SIZE

1073741824.0

In [36]:
frame.waterfall.container._calc_selection_size()

20468

In [37]:
5*2**26

335544320

In [59]:
fchans = 2**26
tchans = 5
df = -2.7939677238464355 * u.Hz
dt = 18.253611008 * u.s
fch1 = 19626.464842353016 * u.MHz
noise_std = 0.05 # Gaussian standard deviation

# Generate the frame.
frame = stg.Frame(fchans=fchans,
                  tchans=tchans,
                  df=df,
                  dt=dt,
                  fch1=fch1)
frame.get_waterfall()

Waterfall data: /home/bryanb/setigen/setigen/assets/sample.fil

In [60]:
frame.waterfall.container.MAX_DATA_ARRAY_SIZE, frame.waterfall.container._calc_selection_size()

(1073741824.0, 1342177260)

In [61]:
frame.waterfall.container.selection_shape

(5, 1, 67108864)

In [62]:
5*2**26*32

10737418240

In [63]:
frame.waterfall.container._n_bytes

4

In [64]:
frame.waterfall.container.t_stop - frame.waterfall.container.t_start

5

In [65]:
(frame.waterfall.container.f_stop - frame.waterfall.container.f_start) / abs(frame.waterfall.container.header['foff'])

67108863.0

In [66]:
frame.waterfall.container.header['foff']

-2.7939677238464355e-06

In [68]:
frame.waterfall.container.f_stop, frame.waterfall.container.f_start

(19626.464842353016, 19438.964845146984)

In [70]:
fchans*5*4

1342177280

In [80]:
fchans = 2**26
tchans = 5
df = -2.7939677238464355 * u.Hz
dt = 18.253611008 * u.s
fch1 = 19626.464842353016 * u.MHz
noise_std = 0.05 # Gaussian standard deviation

# Generate the frame.
frame = stg.Frame(fchans=fchans,
                  tchans=tchans,
                  df=df,
                  dt=dt,
                  fch1=fch1)
frame._update_waterfall()
frame.save_fil('example.fil')

/mnt_home/bryanb/setigen_development/example.fil


In [72]:
INFILE

'/datax/scratch/bbrzycki/data/tabby.1.fil'

In [75]:
import os
fn = 'test.txt'
os.path.isabs(fn)

False

In [76]:
os.path.abspath(fn)

'/mnt_home/bryanb/setigen_development/test.txt'

In [111]:
r'''
Synthesize a Filterbank file.
Motivation: https://github.com/UCBerkeleySETI/turbo_seti/issues/206
'''

import time
from astropy import units as u
import setigen as stg
from blimpy import Waterfall


DIR = '/datax/scratch/bbrzycki/data/'
INFILE = '/datax/scratch/bbrzycki/data/tabby.1.fil'
MAX_DRIFT = 4
MIN_SNR = 25
arg_n_signals = 1
 
    
def gen_fil(arg_path, n_signals):
    r''' Generate a Filterbank file '''

    assert n_signals > 0 and n_signals < 5
    
    # Define parameters.
    
#     fchans = 134217728
    fchans = 2**27
    #fchans = 1342177
    tchans = 5
    df = -2.7939677238464355e-06 * u.Hz
    dt = 18.253611008 * u.s
    fch1 = 19626.464842353016 * u.MHz
    noise_std = 0.05 # Gaussian standard deviation
    
    sig_snr_1 = 100.0
    sig_width_1 = 1.1 * u.Hz
    drate_1 = 1.6 * u.Hz/u.s
    f_start_1 = 0

    sig_snr_2 = 200.0
    sig_width_2 = 1.2 * u.Hz
    drate_2 = 1.3 * u.Hz/u.s
    f_start_2 = fchans * 0.1

    sig_snr_3 = 300.0
    sig_width_3 = 1.3 * u.Hz
    drate_3 = 2.6 * u.Hz/u.s
    f_start_3 = fchans * 0.2

    sig_snr_4 = 400.0
    sig_width_4 = 1.4 * u.Hz
    drate_4 = 3.2 * u.Hz/u.s
    f_start_4 = fchans * 0.3
    
    # Generate the frame.
    frame = stg.Frame(fchans=fchans,
                      tchans=tchans,
                      df=df,
                      dt=dt,
                      fch1=fch1)
    
    # Add noise.
    frame.add_noise(x_mean=0, x_std=noise_std, noise_type='gaussian')

    # Add signal 1.
    signal_intensity = frame.get_intensity(snr=sig_snr_1)
    frame.add_constant_signal(f_start=frame.get_frequency(f_start_1),
                              drift_rate=drate_1,
                              level=signal_intensity,
                              width=sig_width_1,
                              f_profile_type='gaussian')
    
    # Add signal 2.
    if n_signals > 1:    
        signal_intensity = frame.get_intensity(snr=sig_snr_2)
        frame.add_constant_signal(f_start=frame.get_frequency(f_start_2),
                                  drift_rate=drate_2,
                                  level=signal_intensity,
                                  width=sig_width_2,
                                  f_profile_type='gaussian')
    
    # Add signal 3.
    if n_signals > 2:    
        signal_intensity = frame.get_intensity(snr=sig_snr_3)
        frame.add_constant_signal(f_start=frame.get_frequency(f_start_3),
                                  drift_rate=drate_3,
                                  level=signal_intensity,
                                  width=sig_width_3,
                                  f_profile_type='gaussian')
    
    # Add signal 4.
    if n_signals > 3:    
        signal_intensity = frame.get_intensity(snr=sig_snr_4)
        frame.add_constant_signal(f_start=frame.get_frequency(f_start_4),
                                  drift_rate=drate_4,
                                  level=signal_intensity,
                                  width=sig_width_4,
                                  f_profile_type='gaussian')
    
    # Save Filterbank file.
#     frame.get_waterfall()
#     frame.waterfall.container.MAX_DATA_ARRAY_SIZE = frame.waterfall.file_size_bytes
    frame.save_fil(arg_path, max_load=8)
    
    print(len(sigproc.generate_sigproc_header(frame.waterfall)))
    print(frame.waterfall.container.idx_data)
    print(frame.waterfall.file_size_bytes)


t1 = time.time()
gen_fil(INFILE, n_signals=1)
t2 = time.time()
print('gen_fil elapsed time = {:.1f}s'.format(t2 - t1))
wf = Waterfall(INFILE)
wf.info()


352
352
2684354560.0
gen_fil elapsed time = 68.8s

--- File Info ---
      machine_id :                               20
    telescope_id :                                6
         src_raj :                         17:47:15
         src_dej :                     -28:22:59.16
        az_start :                              0.0
        za_start :                              0.0
       data_type :                                1
            fch1 :           19626.464842353016 MHz
            foff :      -2.7939677238464356e-12 MHz
          nchans :                        134217728
          nbeams :                                1
           ibeam :                               -1
           nbits :                               32
   tstart (ISOT) :          2018-12-13T17:12:37.000
    tstart (MJD) :               58465.717094907406
           tsamp :                     18.253611008
            nifs :                                1
     source_name :                        Synth

In [86]:
frame.waterfall.file_size_bytes

1342177280.0

In [112]:
2684354560.0/1e9

2.68435456

In [89]:
from blimpy.io import sigproc
sigproc.generate_sigproc_header(wf)

b'\x0c\x00\x00\x00HEADER_START\n\x00\x00\x00machine_id\x14\x00\x00\x00\x0c\x00\x00\x00telescope_id\x06\x00\x00\x00\x07\x00\x00\x00src_raj\x00\x00\x00\x00\xd8S\x05A\x07\x00\x00\x00src_dej=\n\xd7\xa3L:\x11\xc1\x08\x00\x00\x00az_start\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00za_start\x00\x00\x00\x00\x00\x00\x00\x00\t\x00\x00\x00data_type\x01\x00\x00\x00\x04\x00\x00\x00fch1\x00$\xfa\xbf\x9d*\xd3@\x04\x00\x00\x00foff\xfa~j\xbct\x93\x88\xbd\x06\x00\x00\x00nchans\x00\x00\x00\x04\x06\x00\x00\x00nbeams\x01\x00\x00\x00\x05\x00\x00\x00ibeam\xff\xff\xff\xff\x05\x00\x00\x00nbits \x00\x00\x00\x06\x00\x00\x00tstart\xee\x04q\xf26\x8c\xec@\x05\x00\x00\x00tsamp\xfeC\xa9\xa6\xec@2@\x04\x00\x00\x00nifs\x01\x00\x00\x00\x0b\x00\x00\x00source_name\t\x00\x00\x00Synthetic\x0b\x00\x00\x00rawdatafile\t\x00\x00\x00Synthetic\n\x00\x00\x00HEADER_END'

In [90]:
len(sigproc.generate_sigproc_header(wf))

352

In [92]:
wf.container.idx_data

352

In [98]:
len(sigproc.generate_sigproc_header(frame.waterfall))

352

In [99]:
frame.waterfall.container.idx_data

394

In [108]:
import psutil
stats = psutil.virtual_memory()  # returns a named tuple
available = getattr(stats, 'available')
print(available)

226495070208


In [109]:
available/1e9

226.495070208